In [3]:
#Loading of the necessary libraries
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

In [4]:
#will give your pc a unique id stored in config.txt, please do not remove this file once created
from library.get_id import get_or_create_unique_id

my_id = get_or_create_unique_id()
print("my ID:", my_id)

my ID: 5b41ba5a-9fd4-4049-a2c8-2a92a73f1450


In [7]:
#will try and get the most appropriate device for training
from library.get_device import get_device

device = get_device(no_print=False)

Using DirectML device


In [8]:
#Data preprocessing & augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
#Data preprocessing & augmentation for testing
test_transform =transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
#loading of the dataset
data_dir = "./kaggle/input/brain-tumor/4 classes"
dataset = ImageFolder(root=data_dir, transform=train_transform)

In [12]:
# Split dataset (70-15-15 split)
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [13]:
#assign the preprocessing to the different sets
val_dataset.dataset.tranform = test_transform
test_dataset.dataset.transform = test_transform

In [14]:
# Define CNN Model
class BrainTumorCNN(nn.Module):
    def __init__(self, num_classes=4, dropout=0.5):
        super(BrainTumorCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [15]:
#Initialize the hyperparameters:
class HyperParameters():
    def __init__(self,
                 learning_rate = 0.001,
                 batch_size = 32,
                 epochs = 10,
                 dropout = 0.5,
                 optimizer_type = "adam"
                 ) -> None:
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.dropout = dropout
        self.optimizer_type = optimizer_type
        
    def build_model(self, device):
        return BrainTumorCNN(num_classes=4, dropout=self.dropout).to(device)

    def build_optimizer(self, model):
        if self.optimizer_type == "adam":
            return optim.Adam(model.parameters(), lr=self.learning_rate)
        elif self.optimizer_type == "sgd":
            return optim.SGD(model.parameters(), lr=self.learning_rate, momentum=0.9)
        else:
            raise ValueError(f"Unknown optimizer type: '{self.optimizer_type}'. "
                         f"Supported types: 'adam', 'sgd'.")

    def build_criterion(self):
        return nn.CrossEntropyLoss()

In [16]:
#Initialize the model, criterion, optimizer:
hyperParameters = HyperParameters()
model = hyperParameters.build_model(device)
criterion = hyperParameters.build_criterion()
optimizer = hyperParameters.build_optimizer(model)

In [17]:
#assigns the loaders to the sets
train_loader = DataLoader(train_dataset, batch_size=hyperParameters.batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=hyperParameters.batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=hyperParameters.batch_size, shuffle=False, num_workers=2)

In [17]:
#resets the losses & accuracies
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []
timings =  []

#generates a unique id for the run
from library.get_id import get_unique_run_id
run_id = get_unique_run_id()

#trains & measures the model
for epoch in range(hyperParameters.epochs):
    start_time = time.time()  # start timer
    # Training Phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Validation Phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_acc = 100 * correct / total
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    epoch_time = time.time() - start_time
    timings.append(epoch_time)
    print(f"Epoch [{epoch+1}/{hyperParameters.epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Time: {epoch_time:.2f}s")

c:\Users\rouer\anaconda3\Lib\site-packages\torch\optim\adam.py:534: UserWarning: The operator 'aten::lerp.Scalar_out' is not currently supported on the DML backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at C:\__w\1\s\pytorch-directml-plugin\torch_directml\csrc\dml\dml_cpu_fallback.cpp:17.)
  torch._foreach_lerp_(device_exp_avgs, device_grads, 1 - beta1)


Epoch [1/10], Train Loss: 1.6757, Train Acc: 44.90%, Val Loss: 0.9575, Val Acc: 59.70%, Time: 19.87s
Epoch [2/10], Train Loss: 0.8456, Train Acc: 66.77%, Val Loss: 0.6678, Val Acc: 75.22%, Time: 19.40s
Epoch [3/10], Train Loss: 0.5576, Train Acc: 78.73%, Val Loss: 0.5728, Val Acc: 76.72%, Time: 18.99s
Epoch [4/10], Train Loss: 0.4123, Train Acc: 85.37%, Val Loss: 0.5527, Val Acc: 81.03%, Time: 17.53s
Epoch [5/10], Train Loss: 0.2906, Train Acc: 89.94%, Val Loss: 0.5829, Val Acc: 80.39%, Time: 19.38s
Epoch [6/10], Train Loss: 0.2186, Train Acc: 92.16%, Val Loss: 0.6018, Val Acc: 82.54%, Time: 19.35s
Epoch [7/10], Train Loss: 0.1513, Train Acc: 94.69%, Val Loss: 0.6736, Val Acc: 81.90%, Time: 19.55s
Epoch [8/10], Train Loss: 0.1141, Train Acc: 96.17%, Val Loss: 0.6546, Val Acc: 84.48%, Time: 19.48s
Epoch [9/10], Train Loss: 0.0870, Train Acc: 97.19%, Val Loss: 0.6234, Val Acc: 84.05%, Time: 19.37s
Epoch [10/10], Train Loss: 0.0483, Train Acc: 98.34%, Val Loss: 0.7572, Val Acc: 81.68%, Ti

In [ ]:
#preparing the data: storing the hyperparameters's values
results = {
        'computer_id': my_id,
        'learning_rate': hyperParameters.learning_rate,
        'batch_size': hyperParameters.batch_size,
        'epochs': hyperParameters.epochs,
        'dropout': hyperParameters.dropout,
        'optimizer_type': hyperParameters.optimizer_type,
    }

In [ ]:
#preparing the data: training of the model & storing results
from library.training import train_model

results = results | train_model(model, train_loader, val_loader, criterion, optimizer, device, hyperParameters)

c:\Users\rouer\anaconda3\Lib\site-packages\torch\optim\adam.py:534: UserWarning: The operator 'aten::lerp.Scalar_out' is not currently supported on the DML backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at C:\__w\1\s\pytorch-directml-plugin\torch_directml\csrc\dml\dml_cpu_fallback.cpp:17.)
  torch._foreach_lerp_(device_exp_avgs, device_grads, 1 - beta1)


In [ ]:
#preparing the data: testing of the model & storing results
from library.evaluation import evaluate

results = results | evaluate(model, test_loader, device)

In [27]:
from library.save_results import save_training_results_to_csv

save_training_results_to_csv(results)

'training_results.csv'

In [ ]:
# Save the Model
# torch.save(model.state_dict(), "brain_tumor.pth")

In [ ]:
# Plot Training and Validation Metrics
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("loss over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label="Train Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.title("Accuracy over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()